# 假设

全眼的像差，是最终对视网膜产生影响的原因。那么使用佩戴OK镜之前、或者佩戴短期后的像差数据，是否可能预测出远期的眼轴长或者是屈光状态呢？

## 已知的缺陷

全眼像差受到多方面的影响：

* 瞳孔大小；
* 调节状态；
* 测量时间，OK镜佩戴后，白天的角膜形态是否会逐渐变化，导致像差随着时间改变。



# 数据

来自于[Predictive factors associated with axial length growth and myopia progression in orthokeratology ](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6561598/ )

该文献带有excel数据，共有7个sheet，分别是：

* age, sex, visual acuity：年龄，性别，视力，其中视力用LogMAR，包含了未矫正和最佳矫正视力。
* AXL：眼轴长，用IOL master测量了中央，鼻侧30度，颞侧30度
  >AXL measurement with IOLMaster (Carl Zeiss, Jena, Germany) in central, N30, and T30 gazes
* CR：散瞳验光，用WAM-5500测量了中央，鼻侧30度，颞侧30度
  >cycloplegic refraction; autorefraction (WAM-5500; Shigiya Machinery Works Ltd., Hiroshima, Japan) in central, 30° nasal (N30), and 30° temporal (T30) gazes under cycloplegia
* MR：
  >manifested refraction
* specular microscopy：不知为何，测量了角膜内皮细胞计数。
  >evaluation of the corneal endothelium via noncontact specular microscopy (SP-8000; Konan Medical, Nishinomiya, Japan). 
* aberrometer：像差，给了高阶的Zernike系数。
  >wavefront assessment for a 6-mm pupil using a WASCA aberrometer (Carl Zeiss, Jena, Germany) following pupil dilation using a mixture of 0.5% phenylephrine and 0.5% tropicamide (Mydrin-P; Santen Pharmaceutical, Osaka, Japan)
* pentacam：角膜地形图。
  很遗憾，这里面不是raw data，只有Pre和12mo的K1, K2
* orbscan II：角膜地形图。
  也不是角膜地形图的原始数据，但除了Kmin, Kmax,还有Central corneal thickness, 3-mm-zone irregularity, 5-mm-zone irregularity

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os

%matplotlib inline

## 数据清洗


In [21]:
data_file=os.path.join('data',"pone.0218140.s001.xlsx")
AXL=pd.read_excel(data_file,sheet_name="AXL")
CR=pd.read_excel(data_file,sheet_name="CR ")
aberrometer=pd.read_excel(data_file,sheet_name="aberrometer")
cornea=pd.read_excel(data_file,sheet_name="orbscan II")

,Patient,"OD1, OS2",Pre C AXL,Pre N AXL,Pre T AXL,12mo C AXL,12mo N AXL,12mo T AXL,Pre C-N,Pre C-T,12mo C-N,12mo C-T,delta C-N_12mo,delta C-T_12mo,delta C_12mo,delta N_12mo,delta T_12mo
0,#1,1.0,23.570000,21.760000,21.700000,23.610000,22.160000,21.750000,1.810000,1.870000,1.450000,1.860000,0.360000,-0.050000,0.040000,0.400000,0.050000
1,NaN,2.0,23.460000,21.560000,21.750000,23.230000,22.010000,21.830000,1.900000,1.710000,1.220000,1.400000,0.680000,0.500000,-0.230000,0.450000,0.080000
2,#2,1.0,24.200000,22.900000,22.760000,24.590000,23.310000,23.320000,1.300000,1.440000,1.280000,1.270000,0.020000,0.030000,0.390000,0.410000,0.560000
3,NaN,2.0,24.090000,22.750000,22.690000,24.430000,23.060000,23.010000,1.340000,1.400000,1.370000,1.420000,-0.030000,-0.080000,0.340000,0.310000,0.320000
4,#3,1.0,24.230000,23.770000,23.320000,24.630000,24.130000,23.690000,0.460000,0.910000,0.500000,0.940000,-0.040000,-0.480000,0.400000,0.360000,0.370000
5,NaN,2.0,24.110000,23.700000,23.380000,24.500000,24.050000,23.480000,0.410000,0.730000,0.450000,1.020000,-0.040000,-0.610000,0.390000,0.350000,0.100000
6,#4,1.0,23.070000,22.570000,21.970000,23.730000,23.240000,22.540000,0.500000,1.100000,0.490000,1.190000,0.010000,-0.690000,0.660000,0.670000,0.570000
7,NaN,2.0,22.960000,22.640000,22.120000,23.610000,22.960000,22.660000,0.320000,0.840000,0.650000,0.950000,-0.330000,-0.630000,0.650000,0.320000,0.540000
8,#5,1.0,24.900000,24.400000,24.100000,25.230000,24.290000,23.930000,0.500000,0.800000,0.940000,1.300000,-0.440000,-0.800000,0.330000,-0.110000,-0.170000
9,NaN,2.0,24.860000,24.560000,24.260000,25.260000,24.220000,23.950000,0.300000,0.600000,1.040000,1.310000,-0.740000,-1.010000,0.400000,-0.340000,-0.310000


数据清洗：



In [5]:
df.columns

Index(['Patient', 'Sex (male = 1, female = 2', 'OD1, OS2', 'Age', 'log UCVA',
       'log BCVA'],
      dtype='object')

In [6]:
# target_df=df["Spherical Diopter"].where(df["Right Eye"]=="12 months").dropna()
# sns.distplot(target_df)
